In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy    
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 91 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.14).
0 upgraded, 0 newly installed, 0 to remove and 91 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch
!pip install konlpy

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

torch.manual_seed(123)
np.random.seed(123)

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

from google.colab import drive
drive.mount('/content/drive')\

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qv4oq1sr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qv4oq1sr
using cached model
using cached model
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 128
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 100
learning_rate = 3e-5

In [ ]:
# 학습용 데이터셋 불러오기
import pandas as pd
from konlpy.tag import Mecab
train = pd.read_csv('/content/drive/My Drive/mulcamnlp2021/datain/train.csv')
test = pd.read_csv('/content/drive/My Drive/mulcamnlp2021/datain/test.csv')

# 데이터 전처리
train=train[['과제명', '요약문_연구내용','label']]
test=test[['과제명', '요약문_연구내용']]
train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)

# 대표 질병 5개 추출
train['data']=train['과제명']+train['요약문_연구내용']
test['data']=test['과제명']+test['요약문_연구내용']
def clean_text(sents):
    sents_clean = []
    for sent in sents:
        sents_clean.append(re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]", " ", sent))
    return sents_clean


def tokenize(docs):
  mecab = Mecab()
  res=[]
  for doc in docs:
    tokenlist = mecab.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['NNG', 'NNP', 'NNB', 'NR']:
        temp.append(w[0])
    res.append(' '.join(temp))
  return res

train['data'] = tokenize(clean_text(train['data']))
test['data'] = tokenize(clean_text(test['data']))

# Train / Test set 분리
from sklearn.model_selection import train_test_split
trains, tests = train_test_split(train, test_size=0.2, random_state=123)
print("train shape is:", len(trains))
print("test shape is:", len(tests))


# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i]) for i in dataset['data']]
        self.labels = [np.int32(i) for i in dataset['label']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        
data_train = BERTDataset(trains, 3, 2, tok, max_len, True, False)
data_test = BERTDataset(tests, 3, 2, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

train shape is: 139443
test shape is: 34861
using cached model


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 46, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
# del train, test, data_train, data_test, trains, tests
del train, data_train, data_test, trains, tests

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
highest_acc = 0
patience = 0

# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    #테스트 문장 예측
    
    model.eval() # 평가 모드로 변경
    
    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))

    if test_acc - highest_acc >= 0.01:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, '/content/drive/My Drive/mulcamnlp2021/torchckpt/model.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


epoch 1 batch id 1 loss 3.8252406120300293 train acc 0.0234375
epoch 1 batch id 101 loss 3.2361252307891846 train acc 0.2130259900990099
epoch 1 batch id 201 loss 2.231053352355957 train acc 0.4992615049751244
epoch 1 batch id 301 loss 1.4669082164764404 train acc 0.6032495847176079
epoch 1 batch id 401 loss 1.3117352724075317 train acc 0.6576917082294265
epoch 1 batch id 501 loss 1.3120595216751099 train acc 0.6892776946107785
epoch 1 batch id 601 loss 0.9133090376853943 train acc 0.7112884775374376
epoch 1 batch id 701 loss 1.0392524003982544 train acc 0.7265513552068473
epoch 1 batch id 801 loss 0.9304847717285156 train acc 0.7379545099875156
epoch 1 batch id 901 loss 0.923143744468689 train acc 0.7466443534961155
epoch 1 batch id 1001 loss 0.6584336161613464 train acc 0.7539413711288712

epoch 1 train acc 0.7596899453588775


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8823529411764658
current patience: 0
************************************************************************************


epoch 2 batch id 1 loss 1.1076102256774902 train acc 0.734375
epoch 2 batch id 101 loss 0.6918722987174988 train acc 0.8230971534653465
epoch 2 batch id 201 loss 0.7229592204093933 train acc 0.8268034825870647
epoch 2 batch id 301 loss 0.551899254322052 train acc 0.8268791528239202
epoch 2 batch id 401 loss 0.7899765372276306 train acc 0.8296056733167082
epoch 2 batch id 501 loss 0.7412642240524292 train acc 0.8320234530938124
epoch 2 batch id 601 loss 0.4149633049964905 train acc 0.8351835482529119
epoch 2 batch id 701 loss 0.6622830033302307 train acc 0.8370296897289586
epoch 2 batch id 801 loss 0.6959438920021057 train acc 0.8388830368289638
epoch 2 batch id 901 loss 0.6231672167778015 train acc 0.8403596698113207
epoch 2 batch id 1001 loss 0.49918535351753235 train acc 0.8419549200799201

epoch 2 train acc 0.8431655030131319


...


epoch 28 test acc 1.0
current patience: 0
************************************************************************************


epoch 29 batch id 1 loss 0.025547953322529793 train acc 0.984375
epoch 29 batch id 101 loss 0.009342765435576439 train acc 0.9931157178217822
epoch 29 batch id 201 loss 0.0056269969791173935 train acc 0.9930037313432836
epoch 29 batch id 301 loss 0.01134889479726553 train acc 0.9930699750830565
epoch 29 batch id 401 loss 0.03156683221459389 train acc 0.9934149002493765
epoch 29 batch id 501 loss 0.0012113861739635468 train acc 0.9935909431137725
epoch 29 batch id 601 loss 0.006763978395611048 train acc 0.993409421797005
epoch 29 batch id 701 loss 0.004724093712866306 train acc 0.9936363231098431
epoch 29 batch id 801 loss 0.009990357793867588 train acc 0.9936992821473158
epoch 29 batch id 901 loss 0.004290229640901089 train acc 0.9936268729189789
epoch 29 batch id 1001 loss 0.023840848356485367 train acc 0.9936625874125874

epoch 29 train acc 0.9936281536697248



epoch 29 test acc 1.0
current patience: 0
************************************************************************************


epoch 30 batch id 1 loss 0.0341605618596077 train acc 0.984375
epoch 30 batch id 101 loss 0.009319574572145939 train acc 0.9939665841584159
epoch 30 batch id 201 loss 0.006309538148343563 train acc 0.9940531716417911
epoch 30 batch id 301 loss 0.02153102681040764 train acc 0.9938486295681063
epoch 30 batch id 401 loss 0.019820481538772583 train acc 0.994096789276808
epoch 30 batch id 501 loss 0.0011880180099979043 train acc 0.9942926646706587
epoch 30 batch id 601 loss 0.005668388679623604 train acc 0.9942413685524126
epoch 30 batch id 701 loss 0.0036351950839161873 train acc 0.9942938659058488
epoch 30 batch id 801 loss 0.00866598729044199 train acc 0.9941576935081149
epoch 30 batch id 901 loss 0.006880796980112791 train acc 0.9941904827968924
epoch 30 batch id 1001 loss 0.016397107392549515 train acc 0.9942479395604396

epoch 30 train acc 0.994129873853211



epoch 30 test acc 1.0
current patience: 0
************************************************************************************


In [ ]:
class BERTDataset1(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i]) for i in dataset['data']]
        self.labels = [np.int32(0) for i in dataset['data']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        #return (self.sentences[i] + (0, ))

    def __len__(self):
        return (len(self.sentences))


test_set = BERTDataset1(test, 3, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

checkpoint = torch.load('/content/drive/My Drive/mulcamnlp2021/torchckpt/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
model.eval()

result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
submission = pd.read_csv('/content/drive/My Drive/mulcamnlp2021/datain/sample_submission.csv')

submission['label'] = np.array(result)

submission.to_csv('/content/drive/My Drive/mulcamnlp2021/datain/kobert_baseline.csv', index=False)